In [46]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.ensemble import (
    ExtraTreesClassifier,
    RandomForestClassifier,
    VotingClassifier,
    StackingClassifier,
)
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [21]:
X_mnist, y_mnist = fetch_openml(
    "mnist_784", return_X_y=True, as_frame=False, parser="auto"
)
X_train, y_train = X_mnist[:50_000], y_mnist[:50_000]
X_valid, y_valid = X_mnist[50_000:60_000], y_mnist[50_000:60_000]
X_test, y_test = X_mnist[60_000:], y_mnist[60_000:]

In [22]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [23]:
estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

Training the RandomForestClassifier(random_state=42)
Training the ExtraTreesClassifier(random_state=42)
Training the LinearSVC(max_iter=100, random_state=42, tol=20)
Training the MLPClassifier(random_state=42)


In [24]:
[estimate.score(X_valid, y_valid) for estimate in estimators]

[0.973, 0.9743, 0.8662, 0.9626]

In [25]:
# we are going to combine them into an ensemble that will make predictions based on the majority
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]

In [26]:
voting_clf = VotingClassifier(named_estimators)
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm_clf',
                              LinearSVC(max_iter=100, random_state=42, tol=20)),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [27]:
# the voting classifier made a clone of each classifier and trained the clones using class indices as labels not the original class names
# to evaluate these clones we need to provide clas indices as well to convert them to class names we can use the LabelEncoder class
voting_clf.score(X_valid, y_valid)

0.9743

In [28]:
encoder = LabelEncoder()
y_valid_encoded = encoder.fit_transform(y_valid)

In [30]:
# in the case of MNIST it is simpler to just convert class names to integers since the digits match the class ids
y_valid_encoded = y_valid.astype(np.int64)

In [31]:
[estimator.score(X_valid, y_valid_encoded) for estimator in voting_clf.estimators_]

[0.973, 0.9743, 0.8662, 0.9626]

In [32]:
# we remove the SVM to see if the performace improves it is possible to remove an estimator by setting it to drop using set_params() method
voting_clf.set_params(svm_clf="drop")

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees_clf',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm_clf', 'drop'),
                             ('mlp_clf', MLPClassifier(random_state=42))])

In [33]:
# this does not update the list of trained estimators though so we can just remove the SVM from the list
svm_clf_trained = voting_clf.named_estimators_.pop("svm_clf")
voting_clf.estimators_.remove(svm_clf_trained)
# now we can evaluate the voting classifier again
voting_clf.score(X_valid, y_valid)

0.9758

In [34]:
# to try it with a soft voting classifer we do not need to retrain the classifer we just need to set voting to soft
voting_clf.voting = "soft"
voting_clf.score(X_valid, y_valid)
# this confirms that hard voting is better in this case

0.9717

In [35]:
# once we are confident that we have found an ensemble we try it on the test set to comapre it to the individual classifiers
voting_clf.voting = "hard"
voting_clf.score(X_test, y_test)

0.9737

In [36]:
[
    estimator.score(X_test, y_test.astype(np.int64))
    for estimator in voting_clf.estimators_
]

[0.9702, 0.9703, 0.9632]

STACKING Ensemble

In [38]:
# we aim to run the individual classifiers to make predicitons on the validation set and create a new training set with the resulting predictions
# each training instance is a vector containing the set of predictions from all classifiers for an image and the target is the image's class we can train a classifier on this new training set
X_valid_predictions = np.empty((len(X_valid), len(estimators)), dtype=object)

for index, estimator in enumerate(estimators):
    X_valid_predictions[:, index] = estimator.predict(X_valid)

In [39]:
X_valid_predictions

array([['3', '3', '3', '3'],
       ['8', '8', '8', '8'],
       ['6', '6', '6', '6'],
       ...,
       ['5', '5', '5', '5'],
       ['6', '6', '6', '6'],
       ['8', '8', '8', '8']], dtype=object)

In [40]:
rnd_forest_blender = RandomForestClassifier(
    n_estimators=200, oob_score=True, random_state=42
)
rnd_forest_blender.fit(X_valid_predictions, y_valid)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [41]:
rnd_forest_blender.oob_score_

0.9731

In [44]:
# this trained a blender and together with the classifiers it forms a stacking ensemble we can evaluate the ensemble on the test set and compare it to the voting classifier
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=object)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

y_pred = rnd_forest_blender.predict(X_test_predictions)
accuracy_score(y_test, y_pred)

0.9702

In [47]:
# the stacking ensemble does not perform as well as the voting classifier now we can use a stacking classifier instead of a blender
# stacking classifier uses k-fold cross-validation so we can join the training set and the validation set
X_train_full, y_train_full = X_mnist[:60_000], y_mnist[:60_000]
# this may take a while becuase it uses k-fold cross-validation with 5 folds so it trains 4 classifers 5 times each on 80% plus one time on the whole training set and then final model is trained on the predictions of the 4 classifiers 25 total models
stack_clf = StackingClassifier(named_estimators, final_estimator=rnd_forest_blender)
stack_clf.fit(X_train_full, y_train_full)

StackingClassifier(estimators=[('random_forest_clf',
                                RandomForestClassifier(random_state=42)),
                               ('extra_trees_clf',
                                ExtraTreesClassifier(random_state=42)),
                               ('svm_clf',
                                LinearSVC(max_iter=100, random_state=42,
                                          tol=20)),
                               ('mlp_clf', MLPClassifier(random_state=42))],
                   final_estimator=RandomForestClassifier(n_estimators=200,
                                                          oob_score=True,
                                                          random_state=42))

In [48]:
stack_clf.score(X_test, y_test)

0.9789